<a href="https://colab.research.google.com/github/osman-mo94/Sarcopenia-NLP-project/blob/main/sarcopenia_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and import packages

In [1]:
!pip install spacy
!pip install negspacy
!python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for negspacy: filename=negspacy-1.0.3-py3-none-any.whl size=10435 sha256=8b1eabbebb7acee19aa044e51e3358292771468791665b4ee530c8d7cb0b20eb
  Stored in directory: /root/.cache/pip/wheels/8a/3d/2e/1a8caae6d94020ced19900d0829ca67c1f4d147b755dfc28ec
Successfully built negspacy
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-01-28 17:57:07.071258: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 15.4 MB/s

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import spacy
from spacy.pipeline import EntityRuler
from negspacy.negation import Negex
from negspacy.termsets import termset
from spacy.tokens import Span
from spacy import displacy
from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


# Create data-frame for patient information

In [3]:
#Create a pandas data-frame to contain patient details and source text
patients_df = pd.DataFrame(columns=["Patient ID", "Age", "Sex", "Ethnicity", "Sarcopenia diagnosis", "Text"])

#View head of data-frame
patients_df.head()

,Patient ID,Age,Sex,Ethnicity,Sarcopenia diagnosis,Text


In [4]:
#Add mock test patients
patients_df.loc[len(patients_df.index)] = [1, 82, "Male", "White British", "Sarcopenia",
                                           "Thank you for referring Mr Smith who attended for a face-to-face assessment at the Belsay Clinic accompanied by his niece today. He gives a history of poor mobility and falls; both these problems are longstanding and indeed he was assessed at the Belsay clinic by my colleague Dr Boyle back in 2019 and had a course of physiotherapy at the time. More recently he has lost confidence, his balance is worse and he has fallen more. He is clear that the lack of mobility is his greatest frustration; he is housebound unless he can be accompanied out of the house by his niece and even then he gets out only in a wheelchair. He thinks that a combination of things are stopping him being more mobile: he feels that he lacks strength, has a fear of falling and also has pain in the front and back of his legs. This is worse at night and keeps him awake at times. His last fall was two months ago. He notes feeling unsteady on standing but on close questioning this sensation did not appear to be consistent with vertigo. He also complains of pain across her shoulders starting in the right arm and going across the shoulders to the left arm. This has been present for about six months and is not related to exertion. It is no worse in the morning and he does not feel particularly stiff. He also describes stabbing pain over his right eye that then migrates over the top of his head. This does not seem to be related to stressful events. He has lost a considerable amount of weight - 8kg from March to August this year, and another five kg since as his weight in clinic today was 71.5 kg. His appetite is not as good as it usually is but he denies nausea or vomiting; he avoids constipation by taking laxatives. He does not complain of toothache, does not choke on food or drink and says that he eats reasonably well. He has seen a dietitian. He complains of occasional breathlessness and cough at night and brings up a small quantity of phlegm but this is little changed. He has not noticed any blood in his stools. He complains of a rash on his legs, more on the right than the left, and that her legs are often cool. He notes that this rash has been present for at least 10 years and I also note recent vascular duplex studies that suggest good arterial flow in the legs. He has urinary frequency and is often not aware of when he needs to go to the loo; he also complains of a few minutes of crampy lower abdominal pain after micturition. Perhaps unsurprisingly he is somewhat low in mood but still enjoys going out and seeing people. He denies suicidal ideation or early morning waking. He admits that he tends to live in the past more nowadays. He does not complain of any subjective memory problems. On examination, he was alert and engaged well with the consultation. There was no jaundice, anaemia, cyanosis, clubbing or lymphadenopathy. There was a confluent discolouration on both lower legs which was cool to the touch, not raised or tender and was not blanching. Similar changes were seen above the knees but looked more petechial in nature. Heart sounds were normal, JVP was not raised and the chest was clear. Abdominal examination revealed a 2cm liver edge but no ascites or masses. There was an old scar noted on his left upper arm where a basal cell carcinoma has been removed previously. He was mildly tender across the shoulders and in the thoracic paraspinal muscles; tenderness was not confined to the spine. He was able to raise her arms above her head. Neurological examination revealed normal tone, power and coordination. Cranial nerve examination was normal with no nystagmus. He was able to rise unaided from a chair quite quickly but was very reluctant to take a step forward. However he could walk a few paces with support from one person albeit unsteadily. There was no bradykinesia or tremor noted. His SarcScreen revealed a SARC-F score of 10/10, and Fried frailty score of 2/5 denoting prefrailty. He was unable to attempt the 3m walk but his maximum grip strength was 22 kg. GDS was 10/15 and MMSE was 28/30. An active stand showed a blood pressure of 141/85 with no significant drop although he was unsteady on standing. His 12 lead electrocardiogram showed atrial fibrillation at 80 per minute with nil else of note. There are clearly a complex set of interlocking problems here. He is rather stronger than he thinks he is on although his balance is poor, I think it is a lack of confidence rather than a lack of strength that is preventing him from mobilising more. He accepts this and is happy for us to refer to physiotherapy for strength and balance training which I think will help build his confidence. His mood is low but not sufficiently low to denote depression and I think if we can improve his mobility, his mood will improve. Some of his chronic back pain is undoubtedly due to the degenerative changes seen on MRI and I understand that surgery is not going to be an option for this. I would be reluctant to change her painkillers much at the moment though given that he is already on antiepileptic medications and so adding in other agents for chronic pain may not help much but might interact with these medications. We discussed the fine balance between benefits and side effects of all of these medications today. What would perhaps be helpful though is to reduce the codeine to 15 mg so that he can try and take this more regularly; he finds that the 30 mg dose makes him feel very unsteady on her feet. Having said all of this, it is clear that the blood tests suggest some issues that require further investigation. His breathlessness on exertion and raised BNP suggest that investigation for LV systolic dysfunction causing heart failure would be beneficial and I note that he has already had an echo requested. His weight loss is quite dramatic and this, together with the admittedly longstanding rash on his legs, anaemia, raised ESR and high platelet counts suggest that there is an inflammatory disorder present. Whether this is autoimmune or due to another aetiology is unclear. I have requested bloods today including repeat U&Es, liver function tests, calcium, myeloma screen, CRP and ESR, autoimmune screen, creatine kinase and iron studies. I have also requested an ultrasound of the abdomen to investigate the liver edge that I could feel; I note that a recent chest x-ray showed normal heart size and clear lung fields. Despite the possible finding of horizontal nystagmus by Community Nursing colleagues I could not find any evidence of this today and I don’t think we need to progress to brain scanning at present. Once we have got the blood and ultrasound results, I will phone Mr Smith again and discuss the best way forward. It may be that if we do not find another cause for his inflammatory condition, a trial of steroids might be warranted as some of the features including the shoulder discomfort could be consistent with polymyalgia rheumatica. I have explained this today but also explained that I think we need to investigate further before diving in with treatment in case this is not the diagnosis. "]

patients_df.loc[len(patients_df.index)] = [2, 89, "Female", "Afrocaribbean", "No Sarcopenia", 
                                           "I saw Mrs Smith for a face-to-face appointment at the Belsay Clinic today; she was accompanied by her son. She gives a history of two falls over the last few months and feels unsteady on her feet. She uses a walking frame at home and a stick when she is out. Neither of the falls were witnessed; her son reports that one occurred outside when she was pushing her walker up a ramp; she fell backwards and hit her head. The second occurred in the house but she does not remember any details; when she came round on the floor she managed to press the community alarm button. She denies any lightheadedness or vertigo, any palpitations or chest pain. Interestingly she had two syncopal episodes many years ago, both of which occurred when she was feeling warm and after eating a substantial meal. She has lost a considerable amount of weight over the last year; in November 2020 her weight was 58 kilos, this fell to 55 kilos in June 2021 and her weight in clinic today was 51.4 kilos. Her appetite has been poor but she is now eating better and is due to see the dietitian next week. She does not report choking on food, has no problems chewing, no dysphagia and no nausea. She does not complain of abdominal pain or problems with her bowels. She has no urinary symptoms either. She reports her mood is not low although she gets occasionally frustrated with her inability to complete activities of daily living. I understand that she had a 24 hour blood pressure monitor at your surgery recently. On examination there was no jaundice, anaemia, clubbing, cyanosis or lymphadenopathy. Thyroid examination was normal with no goitre palpable. The JVP was not raised, there was a trace of ankle oedema, heart sounds were normal and the chest was clear. Abdominal examination was unremarkable with no masses or organomegaly. Neurological examination revealed normal tone, power and coordination, no bradykinesia or tremor; cranial nerve examination including gaze was normal and there was no exophthalmos. Her SarcScreen showed a Sarc-F score of 7/10, and Fried frailty score of 2/5 denoting prefrailty. Weight was 51.4 kilos giving a BMI of 23.8. Grip strength was 13 kilos, walk speed 0.42 m/sec, and MMSE was 26/30. An active stand today showed a lying blood pressure of 204/71, falling to 117/46 when standing without any symptoms. This recovered rapidly within a minute to 174/64 and then fell again to 144/56 at 2 minutes. Twelve lead electrocardiogram showed sinus rhythm at 67 beats per minute with no abnormalities. Blood tests done in October show a haemoglobin of 121 with normal LFTs, and in November her TSH was <0.05 with an elevated T4 of 29.3. eGFR was 44 which is stable. Her falls are probably due to a combination of balance impairment and sarcopenia; she has already seen the domiciliary physiotherapy team and she has exercises to do to improve her strength and balance. These are the most effective intervention for this problem and I have encouraged her to keep going with these. Although her falls have not been witnessed, the one in the house has some features that suggest possible syncope and I think it would be sensible to get a 24 hour electrocardiogram; this has been fitted today. Although poor appetite may explain some of her weight loss, she is hyperthyroid suggesting that her thyroxine does is too high; this in turn may be contributing to both her sarcopenia and weight loss. Could I therefore ask you to reduce her thyroxine dose from 10mcg to 75mcg? "]

patients_df.loc[len(patients_df.index)] = [3, 91, "Male", "Asian British", "No Sarcopenia", 
                                           "Diagnoses: Poor mobility due to chronic pain, low confidence and no previous falls. Does not have sarcopenia. The patient report no chronic pain, Weight loss, anaemia and raised inflammatory markers of unknown aetiology Low mood secondary to poor mobility Breathlessness and elevated BNP awaiting echo Chronic back pain with degenerative changes on MRI Urinary frequency and incontinence The patient shows no difficulty walking.The patient has low muscle bulk.Thank you for referring Mr Smith who attended for a face-to-face assessment at the Belsay Clinic accompanied by his niece today. He gives a history of poor mobility and falls; both these problems are longstanding and indeed he was assessed at the Belsay clinic by my colleague Dr Boyle back in 2019 and had a course of physiotherapy at the time. More recently he has lost confidence, his balance is worse and he has fallen more. He is clear that the lack of mobility is his greatest frustration; he is not housebound unless he can be accompanied out of the house by his niece and even then he gets out only in a wheelchair. He thinks that a combination of things are stopping him being more mobile: he feels that he lacks strength, has a fear of falling and also has pain in the front and back of his legs. This is worse at night and keeps him awake at times. His last fall was two months ago. He notes feeling unsteady on standing but on close questioning this sensation did not appear to be consistent with vertigo. He also complains of pain across her shoulders starting in the right arm and going across the shoulders to the left arm. This has been present for about six months and is not related to exertion. It is no worse in the morning and he does not feel particularly stiff. He also describes stabbing pain over his right eye that then migrates over the top of his head. This does not seem to be related to stressful events. He has lost a considerable amount of weight - 8kg from March to August this year, and another five kg since as his weight in clinic today was 71.5 kg. His appetite is not as good as it usually is but he denies nausea or vomiting; he avoids constipation by taking laxatives. He does not complain of toothache, does not choke on food or drink and says that he eats reasonably well. He has seen a dietitian. He complains of occasional breathlessness and cough at night and brings up a small quantity of phlegm but this is little changed. He has not noticed any blood in his stools. He complains of a rash on his legs, more on the right than the left, and that her legs are often cool. He notes that this rash has been present for at least 10 years and I also note recent vascular duplex studies that suggest good arterial flow in the legs. He has urinary frequency and is often not aware of when he needs to go to the loo; he also complains of a few minutes of crampy lower abdominal pain after micturition. Perhaps unsurprisingly he is somewhat low in mood but still enjoys going out and seeing people. He denies suicidal ideation or early morning waking. He admits that he tends to live in the past more nowadays. He does not complain of any subjective memory problems. On examination, he was alert and engaged well with the consultation. There was no jaundice, anaemia, cyanosis, clubbing or lymphadenopathy. There was a confluent discolouration on both lower legs which was cool to the touch, not raised or tender and was not blanching. Similar changes were seen above the knees but looked more petechial in nature. Heart sounds were normal, JVP was not raised and the chest was clear. Abdominal examination revealed a 2cm liver edge but no ascites or masses. There was an old scar noted on his left upper arm where a basal cell carcinoma has been removed previously. He was mildly tender across the shoulders and in the thoracic paraspinal muscles; tenderness was not confined to the spine. He was able to raise her arms above her head. Neurological examination revealed normal tone, power and coordination. Cranial nerve examination was normal with no nystagmus. He was able to rise unaided from a chair quite quickly but was very reluctant to take a step forward. However he could walk a few paces with support from one person albeit unsteadily. There was no bradykinesia or tremor noted. His SarcScreen revealed a SARC-F score of 10/10, and Fried frailty score of 2/5 denoting prefrailty. He was unable to attempt the 3m walk but his maximum grip strength was 22 kg. GDS was 10/15 and MMSE was 28/30. An active stand showed a blood pressure of 141/85 with no significant drop although he was unsteady on standing. His 12 lead electrocardiogram showed atrial fibrillation at 80 per minute with nil else of note. There are clearly a complex set of interlocking problems here. He is rather stronger than he thinks he is on although his balance is poor, I think it is a lack of confidence rather than a lack of strength that is preventing him from mobilising more. He accepts this and is happy for us to refer to physiotherapy for strength and balance training which I think will help build his confidence. His mood is low but not sufficiently low to denote depression and I think if we can improve his mobility, his mood will improve. Some of his chronic back pain is undoubtedly due to the degenerative changes seen on MRI and I understand that surgery is not going to be an option for this. I would be reluctant to change her painkillers much at the moment though given that he is already on antiepileptic medications and so adding in other agents for chronic pain may not help much but might interact with these medications. We discussed the fine balance between benefits and side effects of all of these medications today. What would perhaps be helpful though is to reduce the codeine to 15 mg so that he can try and take this more regularly; he finds that the 30 mg dose makes him feel very unsteady on her feet. Having said all of this, it is clear that the blood tests suggest some issues that require further investigation. His breathlessness on exertion and raised BNP suggest that investigation for LV systolic dysfunction causing heart failure would be beneficial and I note that he has already had an echo requested. His weight loss is quite dramatic and this, together with the admittedly longstanding rash on his legs, anaemia, raised ESR and high platelet counts suggest that there is an inflammatory disorder present. Whether this is autoimmune or due to another aetiology is unclear. I have requested bloods today including repeat U&Es, liver function tests, calcium, myeloma screen, CRP and ESR, autoimmune screen, creatine kinase and iron studies. I have also requested an ultrasound of the abdomen to investigate the liver edge that I could feel; I note that a recent chest x-ray showed normal heart size and clear lung fields. Despite the possible finding of horizontal nystagmus by Community Nursing colleagues I could not find any evidence of this today and I don’t think we need to progress to brain scanning at present. Once we have got the blood and ultrasound results, I will phone Mr Smith again and discuss the best way forward. It may be that if we do not find another cause for his inflammatory condition, a trial of steroids might be warranted as some of the features including the shoulder discomfort could be consistent with polymyalgia rheumatica. I have explained this today but also explained that I think we need to investigate further before diving in with treatment in case this is not the diagnosis."]

patients_df.head()

,Patient ID,Age,Sex,Ethnicity,Sarcopenia diagnosis,Text
0,1,82,Male,White British,Sarcopenia,Thank you for referring Mr Smith who attended ...
1,2,89,Female,Afrocaribbean,No Sarcopenia,I saw Mrs Smith for a face-to-face appointment...
2,3,91,Male,Asian British,No Sarcopenia,"Diagnoses: Poor mobility due to chronic pain, ..."


In [5]:
#Load patient details (from PIONEER data-set) into data-frame

#Remove mock test patients

# Initialize NLP pipeline

In [6]:
#Initialize nlp pipeline with small core spacy model for english
nlp = spacy.load("en_core_web_sm", exclude=["ner"])

In [7]:
#View components of nlp pipeline
nlp.component_names

['tok2vec', 'tagger', 'parser', 'senter', 'attribute_ruler', 'lemmatizer']

# Create matchers for entity recognition

In [8]:
#Initialize matcher
rb_matcher = Matcher(nlp.vocab)

#Define list of weakness paterns
weakness_pattern = [
                    [{"LEMMA": "weak"}], [{"LOWER": "housebound"}], [{"LOWER": "bedbound"}],
                    [{"LEMMA": "use"}, {"LOWER": "a", "OP": "?"}, {"LEMMA": "walk", "OP": "?"}, {"LOWER": "stick"}],
                    [{"LEMMA": "use"}, {"LOWER": "a", "OP": "?"}, {"LEMMA": "walk", "OP": "?"}, {"LOWER": "zimmer", "OP": "?"}, {"LOWER": "frame"}],
                    [{"LEMMA": "use"}, {"LOWER": "a", "OP": "?"}, {"LEMMA": "walk"}, {"LOWER": "aid", "OP": "?"}],
                    [{"LOWER": "furniture"}, {"LEMMA": "walk"}], [{"LEMMA": "difficult"}, {"LEMMA": "walk"}],
                    [{"LEMMA": "difficult"}, {"LEMMA": "mobilise"}], [{"LEMMA": "difficult"}, {"LEMMA": "stand"}],
                    [{"LEMMA": "difficult"}, {"LOWER": "with", "OP": "?"}, {"LEMMA": "climb", "OP": "?"}, {"LEMMA": "stair"}],
                    [{"LOWER": "cannot"}, {"LEMMA": "climb", "OP": "?"}, {"LEMMA": "stair"}],
                    [{"LOWER": "can't"}, {"LEMMA": "climb", "OP": "?"}, {"LEMMA": "stair"}],
                    [{"LEMMA": "hoist"}, {"LEMMA": "transfer"}], [{"LEMMA": "slow"}, {"LOWER": "up"}],
                    [{"LEMMA": "limit"}, {"LOWER": "mobility"}],  [{"LOWER": "poor"}, {"LOWER": "mobility"}],
                    [{"LEMMA": "need"}, {"LEMMA": "assist"}], [{"LEMMA": "require"}, {"LEMMA": "assist"}],
                    [{"LEMMA": "difficult"}, {"LEMMA": "carry"}], [{"LOWER": "found"}, {"LOWER": "on"}, {"LOWER": "floor"}],
                    [{"LOWER": "long"}, {"LOWER": "lie"}], [{"LEMMA": "lack"}, {"LOWER": "of", "OP": "?"}, {"LOWER": "mobility"}],
                    [{"LEMMA": "lack"}, {"LOWER": "of", "OP": "?"}, {"LEMMA": "strength"}],
                    [{"LEMMA": "mob"}, {"LOWER": "with", "OP": "?"}, {"LOWER": "AO2"}],
                    [{"LEMMA": "mob"}, {"LOWER": "with", "OP": "?"}, {"LOWER": "AO1"}],
                    [{"LEMMA": "transfer"}, {"LOWER": "with", "OP": "?"}, {"LOWER": "AO2"}],
                    [{"LEMMA": "transfer"}, {"LOWER": "with", "OP": "?"}, {"LOWER": "AO1"}],
                    [{"LOWER": "sara stedy"}], [{"LOWER": "sara-stedy"}], [{"LOWER": "rotastand"}],
                    [{"LOWER": "wzf"}], [{"LOWER": "zf"}], [{"LOWER": "4ww"}], [{"LOWER": "3ww"}],
                    [{"LOWER": "delta-frame"}], [{"LOWER": "delta frame"}], [{"LOWER": "poc"}], 
                    [{"LEMMA": "pack"}, {"LOWER": "of", "OP": "?"}, {"LEMMA": "care"}],
                    [{"LEMMA": "raise"}, {"LOWER": "toilet", "OP": "?"}, {"LOWER": "seat"}],
                    [{"LOWER": "bed"}, {"LOWER": "lever"}], [{"LOWER": "skandia"}], [{"LOWER": "mowbray"}],
                    [{"LOWER": "free standing toilet frame"}], [{"LOWER": "etwell trolley"}], 
                    [{"LOWER": "kitchen trolley"}], [{"LOWER": "riser recliner"}], [{"LOWER": "raised chair"}],
                    [{"LOWER": "mobile", "OP": "?"}, {"LOWER": "commode"}], [{"LOWER": "stairlift"}], 
                    [{"LOWER": "stair assessment"}], [{"LOWER": "off legs"}], [{"LOWER": "flof"}],
                    [{"LOWER": "cca alarm"}], [{"LOWER": "pendant alarm"}], 
                    [{"LEMMA": "fracture"}, {"LOWER": "pubic", "OP": "?"}, {"LEMMA": "ramus"}],
                    [{"TEXT": "#"}, {"LOWER": "pubic", "OP": "?"}, {"LEMMA": "ramus"}],
                    [{"TEXT": "#NOF"}], [{"TEXT": "#"}, {"LOWER": "nof"}],
                    [{"TEXT": "#"}, {"LOWER": "neck of femur"}], [{"LEMMA": "fracture"}, {"LOWER": "nof"}],
                    [{"LEMMA": "fracture"}, {"LOWER": "nof"}], [{"LEMMA": "fracture"}, {"LOWER": "neck of femur"}],
                    [{"LEMMA": "struggle"}, {"IS_ALPHA": True, "OP": "?"}, {"LEMMA": "stair"}],
                    [{"LEMMA": "deteriorate"}, {"LEMMA": "mobile"}], [{"LEMMA": "frail"}],
                    [{"LEMMA": "downstair"}, {"LEMMA": "living"}], [{"LOWER": "cga"}],
                    [{"LOWER": "comprehensive geriatric assessment"}], [{"LOWER": "chairbound"}],
                    [{"LEMMA": "decondition"}], [{"LEMMA": "mobile"}, {"LEMMA": "decline"}],
                    [{"LOWER": "has"}, {"LEMMA": "care"}], [{"LOWER": "respite care"}],  
                    [{"LEMMA": "loss"}, {"IS_ALPHA": True, "OP": "?"}, {"LEMMA": "function"}],
                    [{"LEMMA": "difficult"}, {"IS_ALPHA": True, "OP": "?"}, {"LOWER": "adl"}],
                    [{"LEMMA": "depend"}, {"IS_ALPHA": True, "OP": "?"}, {"LOWER": "care"}],
                    [{"LEMMA": "fragile"}], [{"LOWER": "immobile"}], 
                    [{"LEMMA": "reduce"}, {"LEMMA": "strength"}], [{"LEMMA": "limit"}, {"LEMMA": "strength"}], 
                    [{"LOWER": "stuck"}, {"IS_ALPHA": True, "OP": "?"}, {"LOWER": "toilet"}],
                    [{"LEMMA": "loss"}, {"IS_ALPHA": True, "OP": "?"}, {"LOWER": "power"}],
                    [{"LOWER": "can't"}, {"LOWER": "stand"}], [{"LOWER": "cannot"}, {"LOWER": "stand"}],
                    [{"LOWER": "unable"}, {"IS_ALPHA": True, "OP": "?"}, {"LOWER": "stand"}],
                    [{"LOWER": "no"}, {"LEMMA": "rehab"}, {"LOWER": "potential"}],
                    [{"LOWER": "limited"}, {"LEMMA": "rehab"}, {"LOWER": "potential"}],
                    [{"LOWER": "mechanical fall"}]
]

#Define list for loss of muscle bulk
muscle_loss_list = [
                    [{"LOWER": "gaunt"}],
                    [{"LEMMA": "muscle"}, {"LEMMA": "waste"}], [{"LEMMA": "cachexia"}],
                    [{"LEMMA": "muscle"}, {"LEMMA": "atrophy"}], [{"LOWER": "anorexia"}],
                    [{"LEMMA": "quad"}, {"LEMMA": "waste"}], [{"LOWER": "low"}, {"LEMMA": "muscle"}, {"LOWER": "bulk"}],
                    [{"LOWER": "loss"}, {"LOWER": "of"}, {"LEMMA": "quad", "OP": "?"}, {"LOWER": "bulk"}]
]

#Add patternS to matcher
rb_matcher.add("WEAKNESS TERM", weakness_pattern)
rb_matcher.add("MUSCLE BULK LOSS", muscle_loss_list)

In [9]:
#Use a phraseMatcher for Sarcopenia diagnosis and SARC-F
matcher = PhraseMatcher(nlp.vocab)

#Add pattern for Sarcopenia diagnosis
sarcopenia_diagnosis = ["Sarcopenia", "sarcopenia"]
sarcopenia_terms = [nlp(i) for i in sarcopenia_diagnosis]

#Add pattern for SARC-F score
sarcf_list = ["SARC-F", "SARC F", "SARCF", "sarc-f", "sarc f", "Sarc f", "Sarc F", "Sarc-f", "Sarc-F"]
sarcf_terms = [nlp(i) for i in sarcf_list]

#Add to matcher
matcher.add("Sarcopenia", sarcopenia_terms)
matcher.add("SARC-F", sarcf_terms)

# Initialize NER ruler

In [10]:
#Initialize NER ruler

ruler = nlp.add_pipe("entity_ruler", after = "lemmatizer")

In [11]:
#Add weakness patterns to NER ruler
for item in weakness_pattern:
  ruler.add_patterns([{"label": "WEAKNESS TERM", "pattern": item}])

#Add muscle bulk liss patterns to the NER ruler
for item in muscle_loss_list:
  ruler.add_patterns([{"label": "MUSCLE BULK LOSS", "pattern": item}])

#Add sarcopenia diagnosis to NER ruler
for item in sarcopenia_diagnosis:
  ruler.add_patterns([{"label": "SARCOPENIA", "pattern": item}])

#Add SARC-F to NER ruler
for item in sarcf_list:
  ruler.add_patterns([{"label": "SARC-F", "pattern": item}])

# Initialize Negation detection


In [12]:
#Define termset as clinical
ts = termset("en_clinical_sensitive")

#Add negex to nlp pipeline
nlp.add_pipe("negex", config={
    "ent_types":["SARCOPENIA", "MUSCLE BULK LOSS", "WEAKNESS TERM","SARC-F"],
    "neg_termset":ts.get_patterns()
})

#Negation termset needs some modification as it is not fully accurate
ts.remove_patterns({"pseudo_negations": ["no further"]})
ts.add_patterns({"preceding_negations": ["no further"]})
ts.remove_patterns({"preceding_negations": ["educating the patient", "concern for", "history of", "taught the patient", "teach the patient", "h/o", "teaching the patient", "educated the patient", "versus", "supposed", "leads to", "educate the patient"]})

#View updated termset
print(ts.get_patterns())

{'pseudo_negations': ['not able to be', 'not certain if', 'not certain whether', 'not necessarily', 'without any further', 'without difficulty', 'without further', 'might not', 'not only', 'no increase', 'no significant change', 'no change', 'no definite change', 'not extend', 'not cause', 'gram negative', 'not rule out', 'not ruled out', 'not been ruled out', 'not drain', 'no suspicious change', 'no interval change', 'no significant interval change'], 'preceding_negations': ["doesn't", 'if you experience', 'doubt', 'fails to reveal', 'didnt', 'which causes', 'patient was not', "didn't", 'no sign of', 'doesnt', 'no complaints of', 'without signs of', 'rule out', "wasn't", 'couldnt', 'rule him out', 'free of', 'no evidence of', 'without', 'no further', 'negative for', 'rule patient out', 'absence of', 'denying', 'rule her out', 'evaluate for', 'no cause of', 'ruled her out', 'without any reactions or signs of', "can't", 'test for', "couldn't", 'not demonstrate', 'ruled him out', 'ruled 

In [13]:
#Create entity label for negative entities
def add_neg_entities(doc):
    new_ents = []
    for ent in doc.ents:
        # Only check for entity if negex is true
        if ent._.negex:
            #print(ent.label_)
            #new_ent = Span(doc, ent.start, ent.end, label=ent.label, label_="NEG_ENTITY")
            ent.label_="NEG_ENTITY"
            
        new_ents.append(ent)
    
    doc.ents = new_ents
    return doc

# Apply Entity recognition to text column in patients_df

In [14]:
#Create variable for Text column from df
texts = patients_df["Text"]

#Create empty list for entities
entity_list = []

#Iterate over docs, and add entities from these into a "entity_list"
for doc in nlp.pipe(texts):
  add_neg_entities(doc)
  entity_list.append([(ent.text, ent.label_) for ent in doc.ents])

print(entity_list)

[[('poor mobility', 'NEG_ENTITY'), ('lack of mobility', 'WEAKNESS TERM'), ('housebound', 'WEAKNESS TERM'), ('lacks strength', 'WEAKNESS TERM'), ('SARC-F', 'SARC-F'), ('lack of strength', 'WEAKNESS TERM')], [('uses a walking frame', 'WEAKNESS TERM'), ('Sarc-F', 'SARC-F'), ('sarcopenia', 'SARCOPENIA'), ('sarcopenia', 'SARCOPENIA')], [('Poor mobility', 'WEAKNESS TERM'), ('sarcopenia', 'NEG_ENTITY'), ('poor mobility', 'WEAKNESS TERM'), ('low muscle bulk', 'MUSCLE BULK LOSS'), ('poor mobility', 'NEG_ENTITY'), ('lack of mobility', 'WEAKNESS TERM'), ('housebound', 'NEG_ENTITY'), ('lacks strength', 'WEAKNESS TERM'), ('SARC-F', 'SARC-F'), ('lack of strength', 'WEAKNESS TERM')]]


In [15]:
#Create a new column in data-frame for entity-list
patients_df["Entity list"] = entity_list

#View data-frame head
patients_df.head()

,Patient ID,Age,Sex,Ethnicity,Sarcopenia diagnosis,Text,Entity list
0,1,82,Male,White British,Sarcopenia,Thank you for referring Mr Smith who attended ...,"[(poor mobility, NEG_ENTITY), (lack of mobilit..."
1,2,89,Female,Afrocaribbean,No Sarcopenia,I saw Mrs Smith for a face-to-face appointment...,"[(uses a walking frame, WEAKNESS TERM), (Sarc-..."
2,3,91,Male,Asian British,No Sarcopenia,"Diagnoses: Poor mobility due to chronic pain, ...","[(Poor mobility, WEAKNESS TERM), (sarcopenia, ..."


# Prepare data-frame for ML

Before machine learning, we will convert the entity list column into multiple columns (1 column for each feature). Each feature in this data-frame will be used as a predictor variable for our outcome (Sarcopenia).

In [16]:
count_df = pd.DataFrame(
    patients_df.explode("Entity list")
    .pivot_table(
        index = "Patient ID", columns = "Entity list", aggfunc= "size", fill_value= 0 
    )
    .reset_index()
)

count_df.head()

Entity list,Patient ID,"(Poor mobility, WEAKNESS TERM)","(SARC-F, SARC-F)","(Sarc-F, SARC-F)","(housebound, NEG_ENTITY)","(housebound, WEAKNESS TERM)","(lack of mobility, WEAKNESS TERM)","(lack of strength, WEAKNESS TERM)","(lacks strength, WEAKNESS TERM)","(low muscle bulk, MUSCLE BULK LOSS)","(poor mobility, NEG_ENTITY)","(poor mobility, WEAKNESS TERM)","(sarcopenia, NEG_ENTITY)","(sarcopenia, SARCOPENIA)","(uses a walking frame, WEAKNESS TERM)"
0,1,0,1,0,0,1,1,1,1,0,1,0,0,0,0
1,2,0,0,1,0,0,0,0,0,0,0,0,0,2,1
2,3,1,1,0,1,0,1,1,1,1,1,1,1,0,0


In [17]:
#Merge count_df onto patients_df
patients_df = pd.merge(patients_df, count_df, on = "Patient ID")

patients_df.head()

,Patient ID,Age,Sex,Ethnicity,Sarcopenia diagnosis,Text,Entity list,"(Poor mobility, WEAKNESS TERM)","(SARC-F, SARC-F)","(Sarc-F, SARC-F)",...,"(housebound, WEAKNESS TERM)","(lack of mobility, WEAKNESS TERM)","(lack of strength, WEAKNESS TERM)","(lacks strength, WEAKNESS TERM)","(low muscle bulk, MUSCLE BULK LOSS)","(poor mobility, NEG_ENTITY)","(poor mobility, WEAKNESS TERM)","(sarcopenia, NEG_ENTITY)","(sarcopenia, SARCOPENIA)","(uses a walking frame, WEAKNESS TERM)"
0,1,82,Male,White British,Sarcopenia,Thank you for referring Mr Smith who attended ...,"[(poor mobility, NEG_ENTITY), (lack of mobilit...",0,1,0,...,1,1,1,1,0,1,0,0,0,0
1,2,89,Female,Afrocaribbean,No Sarcopenia,I saw Mrs Smith for a face-to-face appointment...,"[(uses a walking frame, WEAKNESS TERM), (Sarc-...",0,0,1,...,0,0,0,0,0,0,0,0,2,1
2,3,91,Male,Asian British,No Sarcopenia,"Diagnoses: Poor mobility due to chronic pain, ...","[(Poor mobility, WEAKNESS TERM), (sarcopenia, ...",1,1,0,...,0,1,1,1,1,1,1,1,0,0
